# Домашнее задание №1

##### Гордеев Никита, БКЛ-182

В качестве источника людских мнений я выбрал игровой сервис Steam. При беглом осмыслении мною обнаружились следующие плюсы этой платформы:

+ компьютерные игры - это веселее и интереснее, чем утюги и смартфоны Huawei (но это субъективно!);
+ Steam - сервис с огромным количеством пользователей, и найти продукты, число отзывов к которым превышает несколько тысяч (причём как отрицательных, так и положительных) - легче лёгкого;
+ легко конструировать ссылки, сразу указывая в них язык рецензий и их характер.

Недостатки тоже имеются:

+ отзывы на медиа-продукты почти всегда чуть больше отдают художественностью, пользователи часто стараются выделиться остроумностью (иронизируют), красивым слогом, коммуницируют с сообществом (например, рефереруют к противоположным мнениям) и т. д. - всё это, предположительно, будет снижать accuracy;
+ словники, составленные по конкретным играм (особенно имеющим большой фандом и насыщенную вселенную), нельзя применять к другим нефраншизным играм, потому что они будут изобиловать уникальной лексикой, - но мы и не будем!
+ на Steam отзывы подгружаются по мере скроллинга страницы - будем пользоваться Selenium;
+ деление по шкале оценки бинарное, а это значит, что смешанные отзывы у нас попадут в одну из категорий.

Работать будем с англоязычными отзывами на игру The Witcher 3: Wild Hunt.

Импортируем нужные модули.

In [124]:
import spacy
import re

from sklearn.metrics import accuracy_score
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from collections import Counter
from termcolor import colored
from IPython.display import Image

Функция, составляющая ссылки на странички с положительными и отрицательными отзывами. На вход берутся язык, на котором должны быть отзывы, и ID интересной нам игры.

In [143]:
def make_links(game_id, language):
    
    pos = 'positivereviews'
    neg = 'negativereviews'
    first_part = 'https://steamcommunity.com/app/'
    second_part = '/?browsefilter=toprated&snr=1_5_100010_&filterLanguage='
    link_pos = first_part + game_id + '/' + pos + second_part + '&p=1'
    link_neg = first_part + game_id + '/' + neg + second_part + '&p=1'
        
    return link_pos, link_neg

Далее работаем с Selenium. Открываем полученные ссылки и прокручиваем страницы вниз, пока не получим нужное количество отзывов. Брать будем по 2000 рецензий для составления словарей и по 100 для тестирования нашего сентимент-анализа. Выкачиваем.

In [144]:
def get_reviews(number, link):

    driver = webdriver.Chrome('./chromedriver')
    driver.get(link)
    ActionChains = webdriver.ActionChains
    actions = ActionChains(driver)
    element = []
    reviews = []
    # Убираем окно с предупреждением о контенте 18+, если есть:
    check_age = WebDriverWait(driver, 10).until(lambda x: x.find_element_by_id('age_gate_btn_continue')) 
    if check_age != []:
        actions.click(check_age)
        actions.perform()
    # Скроллим вниз, пока не получим нужное количество отзывов или не упрёмся в тупик:
    while element == [] and len(reviews) <= number:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        element = driver.find_elements_by_link_text('''поделитесь скриншотом, сделайте видео или начните новое обсуждение!''')
        reviews = driver.find_elements_by_class_name('apphub_CardTextContent')
    for i in range(len(reviews)):
        reviews[i] = reviews[i].get_attribute('innerHTML')
    driver.close()
    
    return reviews

При помощи регулярных выражений убираем всё лишнее, лемматизируем пакетом SpaCy, получаем два списка слов, употреблённых в положительных или отрицательных отзывах.

In [145]:
def get_tokens(responses):
    
    nlp = spacy.load('en', disable=['parser', 'ner'])
    voc = []
    reviews = responses
    for i in range(len(reviews)):
        if re.search('(\t{16})(.+)', reviews[i]):
            review = re.search('(\t{16})(.+)', reviews[i]).group(2)
            review = re.sub('<.*?>', ' ', review)
            review = re.sub('(?:([^-A-Za-z ])|([^a-z]-[^a-z]))', ' ', review).lower()
            doc = nlp(review)
            for token in doc:
                if token.lemma_.isalpha():
                    voc.append(token.lemma_)
                    
    # Эти две строчки, исключающие из списков встретившиеся по одному разу леммы, убили accuracy для отр. отзывов.
    #counter = Counter(voc)
    #voc =  [x for x in xs if counter[x] > 1]

    return voc

Убираем из списков слова, употреблённые в обеих категориях отзывов:

In [146]:
def make_lists(voc_pos, voc_neg):
    
    words_pos = []
    words_neg = []
    for word in voc_pos:
        if word not in voc_neg:
            words_pos.append(word)
    for word in voc_neg:
        if word not in voc_pos:
            words_neg.append(word)
            
    return words_pos, words_neg
    

Предсказываем! Аналогичным способом вычленяем из отзыва список лемм и сравниваем его с двумя полученными словариками. Если слов их положительного словаря больше, чем из отрицательного - значит, отзыв положительный, и наоборот. В случае совпадения числа положительных и отрицательных слова, помечаем отзыв как "neutral" и признаём своё поражение. На выходе угаданное настроение отзывов будет подсвечиваться зелёным, неверное - красным

In [162]:
def predict(tor, test_reviews, words_pos, words_neg):
    
    nlp = spacy.load('en', disable=['parser', 'ner'])
    results = []
    for i in range(len(test_reviews)):
        pos_points = 0
        neg_points = 0
        voc = []
        review = re.search('(\t{16})(.+)', test_reviews[i]).group(2)
        review = re.sub('<.*?>', ' ', review)
        review = re.sub("(?:([^-A-Za-z' ])|([^a-z]-[^a-z]))", ' ', review).lower()
        doc = nlp(review)
        for token in doc:
            if token.lemma_.isalpha():
                voc.append(token.lemma_)
        for j in voc:
            if j in words_pos:
                pos_points += 1
            elif j in words_neg:
                neg_points += 1
        if pos_points > neg_points:
            r = 'positive'
            if r == tor:
                print(colored(r, 'green'))
            else:
                print(colored(r, 'red'))
            results.append(r)
        elif neg_points > pos_points:
            r = 'negative'
            if r == tor:
                print(colored(r, 'green'))
            else:
                print(colored(r, 'red'))
            results.append(r)
        else:
            print(colored('neutral', 'red'))
            results.append('neutral')
            
    return results

Функция подсчёта **accuracy** наших результатов.

In [159]:
def compute_accuracy(tor, results):
    
    reference = []
    for i in range(100):
        reference.append(tor)
    acc = accuracy_score(reference, results)
    
    print('Accuracy score is', acc, 'for', tor, 'reviews.')

Запускаем программу.

In [130]:
game_id = '292030'
language = 'english'
number = 2090

link_pos, link_neg = make_links(game_id, language)
reviews_pos, reviews_neg = get_reviews(number, link_pos), get_reviews(number, link_neg)

In [149]:
voc_pos, voc_neg = get_tokens(reviews_pos[100:]), get_tokens(reviews_neg[100:])
words_pos, words_neg = make_lists(voc_pos, voc_neg)

In [166]:
pos, neg = 'positive', 'negative'

results_pos = predict(pos, reviews_pos[:99], words_pos, words_neg)
print('___________________________________________________________')
results_neg = predict(neg, reviews_neg[:99], words_pos, words_neg)

positive
neutral
neutral
neutral
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
positive
neutral
positive
neutral
positive
positive
neutral
positive
neutral
positive
neutral
neutral
positive
neutral
neutral
positive
positive
positive
neutral
neutral
neutral
positive
neutral
positive
positive
positive
positive
positive
neutral
neutral
neutral
positive
neutral
neutral
positive
neutral
positive
positive
neutral
neutral
neutral
positive
negative
positive
neutral
neutral
positive
neutral
neutral
neutral
positive
positive
neutral
neutral
positive
neutral
positive
positive
neutral
neutral
neutral
neutral
neutral
positive
positive
positive
positive
positive
neutral
neutral
neutral
positive
neutral
neutral
positive
positive
positive
negative
neutral
neutral
neutral
neutral
neutral
neutral
positive
neutral
positive
positive
___________________________________________________________
neutral
negative
neutral
negative
negative
negative
negative
neutral
neutral
positive
neutral


Высчитываем точность.

In [167]:
compute_accuracy(pos, results_pos)
compute_accuracy(neg, results_neg)

Accuracy score is 0.44 for positive reviews.
Accuracy score is 0.47 for negative reviews.


### Каков наш итог?

Как видим, точность предсказания оказалось довольно низкой. Объяснений можно предложить много:

+ специфичность отзывов на медиа-продукты (слэнг и ориентиорованность на самовыражение), их неоднородность (как по длине, так и по содержанию);
+ особенности выбранной нами игры;
+ большая выборка рецензий, как для обучения, так и для тестинга;
+ бинарное деление даже для смешанных отзывов - крайние значения шкал отфильтровывают намного более ярко окрашенные по характеру мнения;

**NB!** Кстати вполне возможно, что в основном неоднозначные отзывы у нас и попали в категорию "neutral"!
+ ну и сам метод довольно грубый, что скрывать...

### Как улучшить?

1. Можно автоматизировать поиск индексов игр, чтобы мы давали на вход не рандомный набор цифр, а название продукта.
2. Раз уж у нас есть возможность выбирать язык рецензий, можно сделать аналогичные функции добычи токенов для русского языка.
3. Можно попробовать учитывать отрицания с помощью отдельной функции. Например, ввести отдельный коэффициент, показывающий, насколько отзыв насыщен отрицательными частицами и учитывать его в предсказаниях.
4. Можно отфильтровывать наименее содержательные отзывы. Допустим, убирать все рецензии, содержащие меньше 10 слов.
5. При помощи Counter'a выделить наиболее частотные маркеры каждого типа отзывов и начислять за их наличие больше баллов, чем за редкие (т. е. придать веса).


# THE END.